In [1]:
#download donut model which is used for prediction
! git clone https://github.com/clovaai/donut.git

Cloning into 'donut'...
remote: Enumerating objects: 273, done.
remote: Total 273 (delta 0), reused 0 (delta 0), pack-reused 273
Receiving objects: 100% (273/273), 62.76 MiB | 9.95 MiB/s, done.
Resolving deltas: 100% (126/126), done.


In [2]:
#install all required frameworks and libraries
!pip install -q datasets sentencepiece tensorboard
!pip install -U torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 torchtext==0.12.0 --extra-index-url https://download.pytorch.org/whl/cu113
!pip install -U pytorch-lightning==1.6.4
!pip install -U transformers==4.11.3
!pip install -U timm==0.5.4
!pip install datasets
!pip install zss==1.1.4
!pip install sconf
!pip install nltk
!pip install pdf2image

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 GB 1.4 MB/s eta 0:00:00:00:0100:04
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 5.0 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 1.9 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 2.7 MB/s eta 0:00:0000:0100:01
  Using cached Pillow-9.5.0-cp38-cp38-manylinux_2_28_x86_64.whl (3.4 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.5/585.5 kB 2.7 MB/s eta 0:00:0000:0100:01
  Using cached torchmetrics-0.11.4-py3-none-any.whl (519 kB)
  Using cached pyDeprecate-0.3.2-py3-none-any.whl (10 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.23.2
    Uninstalling protobuf-4.23.2:
      Successfully uninstalled protobuf-4.23.2
     ━━━━━━━━━━━━━━━━

In [2]:
#for donut we convert pdf to image at first then the image are used 
from pdf2image import convert_from_path
images = convert_from_path("Resume_clz.pdf")
for i,image in enumerate(images):
  image_name=f"{str(i)}.jpg"
  image_pth=f'./data/{image_name}'
  image.save(image_pth, 'JPEG')

In [4]:
import argparse

import gradio as gr
import torch
from PIL import Image

from donut.donut import DonutModel


#if input we want to get question answer  from the image
def demo_process_vqa(input_img, question):
    global pretrained_model, task_prompt, task_name
    input_img = Image.fromarray(input_img)
    user_prompt = task_prompt.replace("{user_input}", question)
    output = pretrained_model.inference(input_img, prompt=user_prompt)["predictions"][0]
    return output

#if input we want to get simple data extraction we use this function
def demo_process(input_img):
    global pretrained_model, task_prompt, task_name
    input_img = Image.fromarray(input_img)
    output = pretrained_model.inference(image=input_img, prompt=task_prompt)["predictions"][0]
    return output


if __name__ == "__main__":
    parser = argparse.ArgumentParser()

    parser.add_argument("--port", type=int, default=None)
    parser.add_argument("--url", type=str, default=None)
    args, left_argv = parser.parse_known_args()

    # cord will just help us extract tabular data or data from cv 
    task_name = "cord"

    task_prompt = f"<s_{task_name}>"
    # image from pdf data we just got using pdf2image
    sample_img_path="/home/peshal/projects/cv_data_extract/data/0.jpg"
    
    example_sample = []
    if args.sample_img_path:
        example_sample.append(args.sample_img_path)
    #path to trained model on our data set is being provided and to train you can use train.py file inside donut folder 
    
    pretrained_model = DonutModel.from_pretrained("/home/peshal/projects/cv_data_extract/content/result/train_cord/test_experiment")

    if torch.cuda.is_available():
        pretrained_model.half()
        device = torch.device("cuda")
        pretrained_model.to(device)

    pretrained_model.eval()

    demo = gr.Interface(
        fn=demo_process_vqa if task_name == "docvqa" else demo_process,
        inputs=["image", "text"] if task_name == "docvqa" else "image",
        outputs="json",
        title=f"Donut 🍩 demonstration for `{task_name}` task",
        examples=[example_sample] if example_sample else None,
    )
    demo.launch(server_name=args.url, server_port=args.port)

AttributeError: 'bytes' object has no attribute '__array_interface__'